In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [ ]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [ ]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [ ]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    y=[]
    x=[]
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
        y.append(yi)
        x.append(xi)
    return y,x

First, let's implement the softmax function. Don't forget numerical stability!

In [ ]:
def softmax(x,logits):

  # logits=x
  y=x-torch.max(logits)
  logits_y=logits-torch.max(logits)
  return torch.exp(y) / torch.sum(torch.exp(logits_y), axis=0)
    ### FILL CODE

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [ ]:
def sgd(w, y_train,x_train, niter):
    nlabels, dim = w.shape
    lr=0.001

    for iter in range(niter):
      loss=softmax(x_train[iter],x_train)
      y_pred= predict(x_train[iter],w)
      grad= x_train[iter].T@(y_train[iter]-y_pred)
      w= w-lr*grad

        ### FILL CODE
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [ ]:
def sigmoid(z):
  return 1/(1+np.exp(-z))
  )
def predict(w, x):
    ## FILL CODE
  z = np.dot(x, w)
  return sigmoid(z)

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [ ]:
def compute_accuracy(w, y_test,x_test):
  c=0
  y_pred=predict(w,x_test)
  for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
      c+=1
  return c/len(y_test)
    ## FILL CODE

In [ ]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
y_train,x_train = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, y_train,x_train, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")